In [1]:
import os
import sys


sys.path.append(os.path.join('..', 'common'))

from ranked_models import statement_ranking
from util import entails, print_knowledge_base, materialized

from itertools import combinations

from datatypes import Formula, Bot, Top, KnowledgeBase, Normally, Literal, Atom
from typing import Set


In [2]:
bot = Bot()            # Falsum
top = Top()            # Verum
m = Literal(Atom('m')) # mamalian red blood cells
v = Literal(Atom('v')) # vertebrate red blood cells
a = Literal(Atom('a')) # avian red blood cells
c = Literal(Atom('c')) # cell membrane
n = Literal(Atom('n')) # nucleus
s = Literal(Atom('s')) # mammalian sickle cells
b = Literal(Atom('b')) # bioconcave shape

In [3]:
def weaken_to_dnf(formulas: Set[Formula], by=0) -> Set[Formula]:
    if by <= 0:
        return formulas
    elif by >= len(formulas):
        return {Formula(Top())}
    disjuncts = combinations(formulas, by)
    formula = Bot()
    for disjunct in disjuncts:
        inner_formula = Top()
        for d in disjunct:
            inner_formula = inner_formula & d
        formula = formula | inner_formula
    return {formula}


In [4]:
def weaken_to_cnf(formulas: Set[Formula], to=0) -> Set[Formula]:
    if to <= 0:
        return formulas
    elif to > len(formulas):
        return {Formula(Top())}
    conjuncts = combinations(formulas, to)
    formulas = set()
    for conjunct in conjuncts:
        formula = Bot()
        for c in conjunct:
            formula = formula | c
        formulas.add(formula)
    return formulas

In [5]:

K={-(m >> v) / bot, -(a >> v) / bot, v / c, v / n, m / -n, -(s >> m) / bot, m / b, s / -b}
rank = statement_ranking(K)
print_knowledge_base(K)

{ m → v, s |~ ¬b, v |~ c, v |~ n, s → m, m |~ b, a → v, m |~ ¬n }


In [6]:
r0 = materialized(rank[0])
print(weaken_to_dnf(r0, 0))
print(weaken_to_dnf(r0, 1))
print(weaken_to_dnf(r0, 2))

{v → n, v → c}
{v → n ∨ v → c}
{⊤}


In [7]:
print(weaken_to_cnf(r0, 2))
print(weaken_to_cnf(r0, 1))
print(weaken_to_cnf(r0, 0))

{v → n ∨ v → c}
{v → n, v → c}
{v → n, v → c}


In [8]:
rinf = materialized(rank[float('inf')])
print(weaken_to_cnf(rinf, 3))
print(weaken_to_cnf(rinf, 2))
print(weaken_to_cnf(rinf, 1))
print(weaken_to_cnf(rinf, 0))

{m → v ∨ a → v ∨ s → m}
{m → v ∨ a → v, a → v ∨ s → m, m → v ∨ s → m}
{m → v, a → v, s → m}
{m → v, a → v, s → m}


In [9]:
def lexicographic_closure_dnf(knowledge_base: KnowledgeBase, statement: Normally):
    rank = statement_ranking(knowledge_base)
    i = 0
    r = len(rank) - (float('inf') in rank)
    if r == 0:
        t = materialized(rank[float('inf')])
        rki = -Top() / Bot()
    else:
        while True:
            t = materialized({statement for k in (*range(i+1, r), float('inf')) for statement in rank[k]})
            j = 0
            n = len(rank[i])
            while True:
                rki = weaken_to_dnf(materialized(rank[i]), j)
                j+=1
                if j > n or not entails(t | rki, -statement.left):
                    break
            i += 1
            if i >= r or not entails(t | rki, -statement.left):
               break
    return entails(t | rki, statement.materialize())


In [10]:
lexicographic_closure_dnf(K, s / -n)

True

In [11]:
lexicographic_closure_dnf(K, m / -b)

False

In [12]:
def lexicographic_closure_cnf(knowledge_base: KnowledgeBase, statement: Normally):
    rank = statement_ranking(knowledge_base)
    i = 0
    r = len(rank) - (float('inf') in rank)
    if r == 0:
        t = materialized(rank[float('inf')])
        rki = -Top() / Bot()
    else:
        while True:
            t = materialized({statement for k in (*range(i+1, r), float('inf')) for statement in rank[k]})
            j = 0
            n = len(rank[i])
            while True:
                rki = weaken_to_cnf(materialized(rank[i]), j+1)
                j+=1
                if j > n or not entails(t | rki, -statement.left):
                    break
            i += 1
            if i >= r or not entails(t | rki, -statement.left):
               break
    return entails(t | rki, statement.materialize())


In [13]:
lexicographic_closure_cnf(K, s/-n)

True

In [14]:
lexicographic_closure_cnf(K, m/-b)

False